<a href="https://colab.research.google.com/github/tset-tset-tset/hoge/blob/master/SpeechBrain_emotion_recognition_wav2vec2_IEMOCAP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ライブラリのインストール・モデルのダウンロード
!pip install speechbrain transformers

# https://huggingface.co/speechbrain/emotion-recognition-wav2vec2-IEMOCAP
from speechbrain.pretrained.interfaces import foreign_class
classifier = foreign_class(source="speechbrain/emotion-recognition-wav2vec2-IEMOCAP", pymodule_file="custom_interface.py", classname="CustomEncoderWav2vec2Classifier")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:379: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2Model: ['project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.codevectors', 'project_q.weight', 'project_hid.bias', 'quantizer.weight_proj.bias', 'project_q.bias']
- This IS expected if you are initializing Wav2Vec2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
#@title WAVE ファイルのアップロード

from google.colab import files
uploaded = files.upload()

Saving test.wav to test.wav


In [3]:
#@title 10 秒ごとに区切って認識

import wave
for file_name in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=file_name, length=len(uploaded[file_name])))

    # WAVファイルを開く
    with wave.open(file_name, 'rb') as wav_file:
        frame_rate = wav_file.getframerate()
        channels = wav_file.getnchannels()
        sample_width = wav_file.getsampwidth()
        frame_count = wav_file.getnframes()
        
        # 10秒ごとにファイルを区切る
        segment_length = 10 * frame_rate
        
        # WAVファイルを読み込む
        for i in range(0, frame_count, segment_length):
            
            # 新しいファイル名を作成
            temp_file_name = 'temp.wav'
            
            # WAVファイルを書き込む
            with wave.open(temp_file_name, 'wb') as new_wav_file:
                
                # WAVファイルのヘッダーを設定
                new_wav_file.setframerate(frame_rate)
                new_wav_file.setnchannels(channels)
                new_wav_file.setsampwidth(sample_width)
                
                # WAVファイルからデータを読み込む
                segment = wav_file.readframes(segment_length)
                
                # WAVファイルにデータを書き込む
                new_wav_file.writeframes(segment)

            # 感情認識
            out_prob, score, index, text_lab = classifier.classify_file(temp_file_name)
            print(i / frame_rate, out_prob, score, index, text_lab, sep='\t')

User uploaded file "test.wav" with length 4800198 bytes
0.0	tensor([[8.5806e-11, 1.6266e-11, 1.0000e+00, 2.0403e-11]])	tensor([1.])	tensor([2])	['hap']
10.0	tensor([[1.7404e-11, 1.2647e-11, 1.0000e+00, 1.0969e-11]])	tensor([1.])	tensor([2])	['hap']
20.0	tensor([[5.5209e-11, 1.0660e-11, 1.0000e+00, 1.0519e-11]])	tensor([1.])	tensor([2])	['hap']
30.0	tensor([[8.4341e-12, 3.9679e-11, 1.0000e+00, 5.9422e-12]])	tensor([1.])	tensor([2])	['hap']
40.0	tensor([[4.2219e-01, 3.1830e-10, 5.7780e-01, 6.3608e-06]])	tensor([0.5778])	tensor([2])	['hap']
50.0	tensor([[9.7243e-11, 8.8573e-12, 1.0000e+00, 3.2734e-11]])	tensor([1.])	tensor([2])	['hap']
60.0	tensor([[2.3195e-06, 1.8573e-10, 1.0000e+00, 6.6914e-11]])	tensor([1.0000])	tensor([2])	['hap']
70.0	tensor([[8.2046e-12, 7.5858e-11, 1.0000e+00, 7.8809e-12]])	tensor([1.])	tensor([2])	['hap']
80.0	tensor([[1.1263e-11, 1.0131e-10, 1.0000e+00, 6.1921e-12]])	tensor([1.])	tensor([2])	['hap']
90.0	tensor([[3.0437e-11, 2.5020e-11, 1.0000e+00, 1.8417e-11]])	